In [1]:
import pandas as pd
import numpy as np
from datetime import datetime 
#import io
#import fbprophet
import matplotlib as plt
import math

In [2]:
holiday = pd.read_csv('TCU_HolidayList.csv',parse_dates=['HolidayDate'])
train_sup = pd.read_csv('Training_SupplementalFields.csv', parse_dates=['BusinessDate'])
train = pd.read_csv('Training.csv', parse_dates=["BusinessDate"])

In [3]:
# Clean up column names
holiday.columns = holiday.columns.str.strip()
train_sup.columns = train_sup.columns.str.strip()
train.columns = train.columns.str.strip()

# Merge Data and Clean Up

In [5]:
# Merge train and train_sup
train_tot = pd.merge(train, train_sup, on=['BusinessDate', 'BranchID'], how='outer') 

# Strip spaces in all monetary values
# Remove commas
train_tot[train_tot.columns[2:]] = (train_tot[train_tot.columns[2:]]
                                    .apply(lambda x:x.str.strip()) # strip white spaces
                                    .apply(lambda x:x.str.replace(pat=',',repl='')) # remove ','
                                    )
# Clean up all columns
# Clean TotalCashUsed to include negative
train_tot['TotalCashUsed'] = (train_tot.TotalCashUsed
                              .str.replace(pat='\(',repl='-') # replace ( with -
                              .str.replace(pat='\)',repl="") # remove )
                              )

# Convert values to float and set NA = 0
train_tot[train_tot.columns[2:]] = (train_tot[train_tot.columns[2:]]
                                   .apply(lambda x:pd.to_numeric(x,errors='coerce'))
                                   .replace(np.nan,0)
                                   )

# Merge Train_tot with Holiday

In [7]:
train_full = pd.merge(train_tot, holiday, left_on = 'BusinessDate', right_on='HolidayDate', how='left').fillna(0)
#train_full['HolidayName'] = train_full['HolidayName'].str.strip()
train_full.info()
#train_full['Holiday'] = train_full['HolidayName'].apply(lambda x: 'Yes' if x.isnull() == False else 'No')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171230 entries, 0 to 171229
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   BusinessDate                 171230 non-null  datetime64[ns]
 1   BranchID                     171230 non-null  int64         
 2   CashOrdersReceived           171230 non-null  float64       
 3   CashBalance_StartOfDay       171230 non-null  float64       
 4   TotalCashUsed                171230 non-null  float64       
 5   CashToVault                  171230 non-null  float64       
 6   CashUnder                    171230 non-null  float64       
 7   CashOver                     171230 non-null  float64       
 8   TotalCashDisbursed           171230 non-null  float64       
 9   TotalCashOpeningBalance      171230 non-null  float64       
 10  TotalCashReceivedAllSources  171230 non-null  float64       
 11  HolidayDate               

In [8]:
# Add Holiday column with Yes or No
train_full['Holiday'] = train_full.apply(lambda row: 'No' if row['HolidayName'] == 0 else 'Yes', axis = 1)
train_full['TotalCashUsed_Holiday'] = train_full.apply(lambda row:0 if row['Holiday'] == 'Yes' else row['TotalCashUsed'],axis=1)

In [20]:
train_full[train_full['Holiday'] =='Yes'].head()

,BusinessDate,BranchID,CashOrdersReceived,CashBalance_StartOfDay,TotalCashUsed,CashToVault,CashUnder,CashOver,TotalCashDisbursed,TotalCashOpeningBalance,TotalCashReceivedAllSources,HolidayDate,HolidayName,Holiday,TotalCashUsed_Holiday
28,2020-07-03,99,0.00,841127.43,0.00,0.0,0.0,0.0,0.0,841127.43,0.0,2020-07-03 00:00:00,Independence Day,Yes,0.0
67,2020-05-25,99,0.00,1277210.34,81158.36,0.0,0.0,0.0,0.0,1277210.34,0.0,2020-05-25 00:00:00,Memorial Day,Yes,0.0
165,2020-02-17,99,0.00,1769224.40,41604.16,0.0,0.0,0.0,0.0,1769224.40,0.0,2020-02-17 00:00:00,Presidents Day,Yes,0.0
193,2020-01-20,99,0.00,2280408.04,47798.31,0.0,0.0,0.0,0.0,2280408.04,0.0,2020-01-20 00:00:00,MLK Day,Yes,0.0
212,2020-01-01,99,1044701.11,1135454.27,-992976.78,0.0,0.0,0.0,0.0,1135454.27,0.0,2020-01-01 00:00:00,New Years Day,Yes,0.0


In [10]:
len(train_full['HolidayDate'].unique())

109

In [11]:
train_full.to_csv('C:/Users/dan/Documents/Kaggle/Crossroads_Datathon/train_full.csv', index = False)

# Reindex

In [12]:
# Date DF
date_df = pd.DataFrame(train_full['BusinessDate'].sort_values().unique())
date_df.shape
date_df.columns = ['Date']

In [13]:
date_df = date_df.reset_index()
date_df

,index,Date
0,0,2010-01-01
1,1,2010-01-02
2,2,2010-01-03
3,3,2010-01-04
4,4,2010-01-05
...,...,...
3832,3832,2020-07-27
3833,3833,2020-07-28
3834,3834,2020-07-29
3835,3835,2020-07-30


In [14]:
# Merge date_df with train_full
branchID_df = pd.merge(train_full, date_df, left_on='BusinessDate', right_on='Date', how='outer')
branchID_df.head()

,BusinessDate,BranchID,CashOrdersReceived,CashBalance_StartOfDay,TotalCashUsed,CashToVault,CashUnder,CashOver,TotalCashDisbursed,TotalCashOpeningBalance,TotalCashReceivedAllSources,HolidayDate,HolidayName,Holiday,TotalCashUsed_Holiday,index,Date
0,2020-07-31,99,0.0,775054.00,-43718.13,0.0,0.00,0.00,1196392.62,1095684.09,875762.53,0,0,No,-43718.13,3836,2020-07-31
1,2020-07-31,108,0.0,1281221.59,64844.13,0.0,0.00,0.00,1063913.35,1554240.66,790894.27,0,0,No,64844.13,3836,2020-07-31
2,2020-07-31,110,0.0,1322202.39,35482.71,0.0,0.00,0.00,1666104.70,1941358.86,1046948.23,0,0,No,35482.71,3836,2020-07-31
3,2020-07-31,111,0.0,1469461.59,63389.48,0.0,521.13,444.11,1835482.60,2561457.52,743563.68,0,0,No,63389.48,3836,2020-07-31
4,2020-07-31,115,0.0,1427862.92,55693.44,0.0,840.21,0.00,924685.51,1701532.24,651856.40,0,0,No,55693.44,3836,2020-07-31


In [15]:
branchID_df['Date_Branch'] = branchID_df['index'].astype(str) + '_' + branchID_df['BranchID'].astype(str)
#branchID_df.head()

,BusinessDate,BranchID,CashOrdersReceived,CashBalance_StartOfDay,TotalCashUsed,CashToVault,CashUnder,CashOver,TotalCashDisbursed,TotalCashOpeningBalance,TotalCashReceivedAllSources,HolidayDate,HolidayName,Holiday,TotalCashUsed_Holiday,index,Date,Date_Branch
0,2020-07-31,99,0.0,775054.00,-43718.13,0.0,0.00,0.00,1196392.62,1095684.09,875762.53,0,0,No,-43718.13,3836,2020-07-31,3836_99
1,2020-07-31,108,0.0,1281221.59,64844.13,0.0,0.00,0.00,1063913.35,1554240.66,790894.27,0,0,No,64844.13,3836,2020-07-31,3836_108
2,2020-07-31,110,0.0,1322202.39,35482.71,0.0,0.00,0.00,1666104.70,1941358.86,1046948.23,0,0,No,35482.71,3836,2020-07-31,3836_110
3,2020-07-31,111,0.0,1469461.59,63389.48,0.0,521.13,444.11,1835482.60,2561457.52,743563.68,0,0,No,63389.48,3836,2020-07-31,3836_111
4,2020-07-31,115,0.0,1427862.92,55693.44,0.0,840.21,0.00,924685.51,1701532.24,651856.40,0,0,No,55693.44,3836,2020-07-31,3836_115


In [16]:
branchID_df.drop(['index','Date'], axis=1,inplace=True)

In [17]:
branchID_df.head()

,BusinessDate,BranchID,CashOrdersReceived,CashBalance_StartOfDay,TotalCashUsed,CashToVault,CashUnder,CashOver,TotalCashDisbursed,TotalCashOpeningBalance,TotalCashReceivedAllSources,HolidayDate,HolidayName,Holiday,TotalCashUsed_Holiday,Date_Branch
0,2020-07-31,99,0.0,775054.00,-43718.13,0.0,0.00,0.00,1196392.62,1095684.09,875762.53,0,0,No,-43718.13,3836_99
1,2020-07-31,108,0.0,1281221.59,64844.13,0.0,0.00,0.00,1063913.35,1554240.66,790894.27,0,0,No,64844.13,3836_108
2,2020-07-31,110,0.0,1322202.39,35482.71,0.0,0.00,0.00,1666104.70,1941358.86,1046948.23,0,0,No,35482.71,3836_110
3,2020-07-31,111,0.0,1469461.59,63389.48,0.0,521.13,444.11,1835482.60,2561457.52,743563.68,0,0,No,63389.48,3836_111
4,2020-07-31,115,0.0,1427862.92,55693.44,0.0,840.21,0.00,924685.51,1701532.24,651856.40,0,0,No,55693.44,3836_115


In [18]:
branchID_df.to_csv('C:/Users/dan/Documents/Kaggle/Crossroads_Datathon/branchID.csv', index=False)